In [37]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, GlobalAveragePooling1D
from tensorflow.keras import backend as K

In [38]:
# Custom convolution block

def conv_layers(x, filters, kernel_size, strides=1, is_train=True):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x) if is_train else x
    x = Activation('relu')(x)
    return x

In [39]:
# Load ECG data

data = pd.read_csv(r"D:\JN\data set\created dataset\ECG\resampled_ecg_data_with_filenames.csv")  # CSV should be (1975, 2048)
data.shape

(1975, 2049)

In [40]:
X = data.iloc[:, 1:].values  # Dropping the first column (index 0)

In [41]:
X.shape

(1975, 2048)

In [42]:
# Reshape for Conv1D: (samples, time_steps, channels)
X = X.reshape((X.shape[0], X.shape[1], 1))  # (1975, 2048, 1)

In [43]:
X.shape

(1975, 2048, 1)

In [44]:
# Build the CNN model
# -------------------------
inputs = Input(shape=(2048, 1))

# 1
layer = conv_layers(inputs, filters=16, kernel_size=15, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 2
layer = conv_layers(layer, filters=16, kernel_size=15, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 3
layer = conv_layers(layer, filters=32, kernel_size=11, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 4
layer = conv_layers(layer, filters=32, kernel_size=11, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 5
layer = conv_layers(layer, filters=64, kernel_size=7, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 6
layer = conv_layers(layer, filters=64, kernel_size=7, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 7
layer = conv_layers(layer, filters=128, kernel_size=3, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 8
layer = conv_layers(layer, filters=128, kernel_size=3, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=2)(layer)
layer = Dropout(0.2, seed=1)(layer)
# Final global pooling to reduce to (1975, 128)
# -------------------------
layer = GlobalAveragePooling1D()(layer)  # Output: (1975, 128)

In [45]:
# Create the model
feature_model = Model(inputs=inputs, outputs=layer)
feature_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 2048, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_24 (Conv1D)                   │ (None, 2048, 16)            │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 2048, 16)            │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_24 (Activation)           │ (None, 2048, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_24 (MaxPooling1D)      │ (None, 1024, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 1024, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_25 (Conv1D)                   │ (None, 1024, 16)            │           3,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 1024, 16)            │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_25 (Activation)           │ (None, 1024, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_25 (MaxPooling1D)      │ (None, 512, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 512, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_26 (Conv1D)                   │ (None, 512, 32)             │           5,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 512, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_26 (Activation)           │ (None, 512, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_26 (MaxPooling1D)      │ (None, 256, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 256, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_27 (Conv1D)                   │ (None, 256, 32)             │          11,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_27               │ (None, 256, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_27 (Activation)           │ (None, 256, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 140,112 (547.31 KB)

 Trainable params: 139,152 (543.56 KB)

 Non-trainable params: 960 (3.75 KB)

In [46]:
# Extract features
features = feature_model.predict(X)  # shape: (1975, 128)

62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step


In [47]:
features.shape

(1975, 128)

In [49]:
# Save to CSV
np.savetxt(r"D:\JN\data set\created dataset\new_extracted_ECG_features.csv", features, delimiter=",")
print("Feature extraction complete. Saved to extracted_ECG_features.csv")

Feature extraction complete. Saved to extracted_ECG_features.csv
